In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
from find_image import FindImage
import os
from dotenv import load_dotenv
import json 

2023-10-21 12:53:36.074148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-21 12:53:36.074186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-21 12:53:36.074210: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-21 12:53:36.079456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 12:53:36.971176: W tensorflow/compiler/

In [2]:
load_dotenv()

True

In [3]:
s3 = FindImage(True, bucket_name=os.getenv("BUCKET_S3"), resource_name="s3")

In [4]:
# Ajustar o caminho para que o jupyter tenha permissão de escrita na pagina
# s3.download_images_training(os.getenv("IMGS_TRAINING"), "pets")

In [5]:
train_dir = '/home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_training/'

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=90, 
    brightness_range=[0.1, 0.7],
    width_shift_range=0.5, 
    height_shift_range=0.5,
    horizontal_flip=True, 
    vertical_flip=True,
    validation_split=0.15,
    preprocessing_function=preprocess_input, # VGG16 preprocessing
    fill_mode='nearest'
)

In [7]:
BATCH_SIZE = 32
train_generator = train_datagen.flow_from_directory(
    train_dir, #diretorio com as img
    target_size=(224, 224),  # Tamanho das imagens (ajuste conforme necessário)
    batch_size=BATCH_SIZE, # Em quantos pacotes/partes eu divido meu dataset 
    class_mode='categorical',  # Modo categórico para múltiplas classes
    subset='training', # Especifica que este é o gerador de treinamento
    shuffle=True,
    seed=42
)

Found 60 images belonging to 15 classes.


In [8]:
vgg = VGG16(input_shape=(224,224,3), include_top = False, weights= 'imagenet', pooling='max')

top_model = vgg.output
top_model = layers.Flatten(name="flatten")(top_model)
top_model = layers.Dense(1024, activation='relu')(top_model)
top_model = layers.BatchNormalization()(top_model)
top_model = layers.Dense(512, activation='relu')(top_model)
top_model = layers.Dropout(0.2)(top_model)
output_layer = layers.Dense(train_generator.num_classes, activation='softmax')(top_model)

model = models.Model(inputs=vgg.input, outputs=output_layer)

2023-10-21 12:53:38.537661: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-21 12:53:38.561037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-21 12:53:38.561258: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_generator,
  batch_size=BATCH_SIZE,
  epochs=30)

Epoch 1/30


2023-10-21 12:53:41.643174: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
2023-10-21 12:53:42.047098: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-21 12:53:42.631190: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-21 12:53:43.197595: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

2/2 [==============================] - 36s 15s/step - loss: 3.5367 - accuracy: 0.0333
Epoch 2/30
2/2 [==============================] - 2s 680ms/step - loss: 3.7630 - accuracy: 0.0333
Epoch 3/30
2/2 [==============================] - 2s 766ms/step - loss: 3.4026 - accuracy: 0.0833
Epoch 4/30
2/2 [==============================] - 2s 768ms/step - loss: 3.3354 - accuracy: 0.0333
Epoch 5/30
2/2 [==============================] - 2s 682ms/step - loss: 3.0736 - accuracy: 0.1000
Epoch 6/30
2/2 [==============================] - 2s 677ms/step - loss: 3.2399 - accuracy: 0.1333
Epoch 7/30
2/2 [==============================] - 2s 677ms/step - loss: 3.3305 - accuracy: 0.0333
Epoch 8/30
2/2 [==============================] - 2s 765ms/step - loss: 3.1069 - accuracy: 0.1333
Epoch 9/30
2/2 [==============================] - 2s 680ms/step - loss: 3.0413 - accuracy: 0.0833
Epoch 10/30
2/2 [==============================] - 2s 764ms/step - loss: 3.0221 - accuracy: 0.1000
Epoch 11/30
2/2 [==============

In [11]:
class_labels = list(train_generator.class_indices.keys())
print(class_labels)

['6531dfe2ebaec4837b4f1f9c', '6531e147ebaec4837b4f1fb6', '6531e1d1ebaec4837b4f1fcb', '6531e32aebaec4837b4f1fe6', '6531e434ebaec4837b4f2001', '6531e4a1ebaec4837b4f2022', '6531e520ebaec4837b4f203b', '6531e677ebaec4837b4f2064', '6531e76febaec4837b4f207f', '6531e7deebaec4837b4f2096', '6531e84cebaec4837b4f20ab', '6531ea71ebaec4837b4f20c5', '6531ead5ebaec4837b4f20da', '6531eb3aebaec4837b4f20ef', '6531f14e6cf94bd6b850ea90']


In [12]:
dict_class_labels = {}
for index, class_label in enumerate(class_labels):
  dict_class_labels[index] = class_label

print(dict_class_labels)

{0: '6531dfe2ebaec4837b4f1f9c', 1: '6531e147ebaec4837b4f1fb6', 2: '6531e1d1ebaec4837b4f1fcb', 3: '6531e32aebaec4837b4f1fe6', 4: '6531e434ebaec4837b4f2001', 5: '6531e4a1ebaec4837b4f2022', 6: '6531e520ebaec4837b4f203b', 7: '6531e677ebaec4837b4f2064', 8: '6531e76febaec4837b4f207f', 9: '6531e7deebaec4837b4f2096', 10: '6531e84cebaec4837b4f20ab', 11: '6531ea71ebaec4837b4f20c5', 12: '6531ead5ebaec4837b4f20da', 13: '6531eb3aebaec4837b4f20ef', 14: '6531f14e6cf94bd6b850ea90'}


In [13]:
class_names_file = 'class_names.json'
with open(class_names_file, 'w') as file:
    json.dump(dict_class_labels, file)

In [16]:
model.save(os.getenv("MODEL_PATH"))

INFO:tensorflow:Assets written to: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/modelv1/assets


INFO:tensorflow:Assets written to: /home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/modelv1/assets
